### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_cirrhosis/GSE163211'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on the information provided, this dataset includes gene expression data

# Data Availability
trait_row = 8  # Based on 'nafld stage'
age_row = 3  # Based on 'age'
gender_row = 4  # Based on 'Sex'

# Data Type Conversion Functions
def convert_trait(value):
    value = value.split(':')[1].strip() if ':' in value else value.strip()
    if value in ['Normal', 'Steatosis', 'NASH_F0', 'NASH_F1_F4']:
        return value
    return None

def convert_age(value):
    value = value.split(':')[1].strip() if ':' in value else value.strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[1].strip() if ':' in value else value.strip()
    if value == 'Female':
        return 0
    elif value == 'Male':
        return 1
    return None

# Save Metadata
save_cohort_info('GSE163211', './preprocessed/Liver_cirrhosis/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract and save it
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Liver_cirrhosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Liver_cirrhosis/trait_data/GSE163211.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Since requires_gene_mapping is False, there is no need to perform any additional actions in this step.


### Step 7: Data Normalization and Merging

In [ ]:

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Liver_cirrhosis/gene_data/GSE163211.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# Debugging: Check the data types in the merged data
print(merged_data.dtypes)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Liver_cirrhosis')

# 4. Save the cohort information, ensuring variable consistency.
save_cohort_info('GSE163211', './preprocessed/Liver_cirrhosis/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Liver_cirrhosis/GSE163211.csv'
    unbiased_merged_data.to_csv(csv_path)
